In [2]:
import torch 
from torch_geometric.datasets import AMiner, Planetoid
from torch_geometric.nn import GCNConv, GATConv, GINConv, MLP




In [39]:
path = r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\metapath2vec\Aminer'
dataset = AMiner(path)
data = dataset[0]
homo = data.to_homogeneous()


In [31]:
path =r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\GCN\data'
dataset = Planetoid(root= path, name='Cora')
cora = dataset[0]

In [35]:
cora.num_nodes

2708

In [36]:
cora.edge_index.shape

torch.Size([2, 10556])

In [32]:
cora.y.shape

torch.Size([2708])

In [33]:
cora.train_mask.shape

torch.Size([2708])

In [24]:
print(data)

HeteroData(
  author={
    y=[246678],
    y_index=[246678],
    num_nodes=1693531,
  },
  venue={
    y=[134],
    y_index=[134],
    num_nodes=3883,
  },
  paper={ num_nodes=3194405 },
  (paper, written_by, author)={ edge_index=[2, 9323605] },
  (author, writes, paper)={ edge_index=[2, 9323605] },
  (paper, published_in, venue)={ edge_index=[2, 3194405] },
  (venue, publishes, paper)={ edge_index=[2, 3194405] }
)


In [25]:
homo = data.to_homogeneous()

In [26]:
print(homo)

Data(edge_index=[2, 25036020], y=[3441217], y_index=[3441217], node_type=[4891819], edge_type=[25036020])


In [33]:
homo.edge_index

tensor([[1697414, 1697415, 1697416,  ..., 1696679, 1696679, 1696679],
        [      0,       1,       2,  ..., 4891816, 4891817, 4891818]])

In [40]:
homo.y.shape


torch.Size([3441217])

In [11]:
homo.y_index


tensor([ 168866, 1327323,     870,  ...,      -1,      -1,      -1])

In [6]:
homo.node_type

tensor([0, 0, 0,  ..., 2, 2, 2])

In [8]:
homo.edge_type

tensor([0, 0, 0,  ..., 3, 3, 3])

In [4]:
print(homo.edge_type.shape, homo.edge_index.shape) 
# edge_type tells us the type of edge it is 
# edge_index tells us which nodes are connected with each other 


torch.Size([25036020]) torch.Size([2, 25036020])


In [30]:
logit = torch.randn(4891819, 7)

In [31]:
homo.y[homo.y != -1]

tensor([0, 2, 5,  ..., 7, 7, 7])

In [32]:
logit[homo.y_index][homo.y != -1]

tensor([[-1.0054,  0.6463, -0.0601,  ...,  0.3903,  1.7832,  0.4792],
        [-1.1563, -1.0221, -0.1909,  ...,  0.9849, -0.4358, -0.0301],
        [ 0.5020, -0.3755, -1.1954,  ...,  0.2668, -0.1184,  0.9632],
        ...,
        [-0.8231, -0.9942,  3.2841,  ..., -0.3404, -0.8767, -0.1286],
        [ 1.4441,  0.2791, -0.1515,  ...,  0.2072,  1.1465,  0.7235],
        [-0.8635,  1.2216, -0.6395,  ...,  0.2325,  0.1296,  0.0270]])

In [29]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, GATConv, GINConv, MLP
import torch_geometric.transforms as T
from sklearn.model_selection import train_test_split

class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        return x

def train(model, optimizer, data, train_mask):
    model.train()
    optimizer.zero_grad()
    if model == GCN:
        out = model(data.x, data.edge_index, data.edge_weight)
    else: 
        out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def evaluate(model, data, mask):
    model.eval()
    logits = model(data.x, data.edge_index)
    loss = F.cross_entropy(logits[mask], data.y[mask])
    pred = logits[mask].max(1)[1]
    correct = pred.eq(data.y[mask]).sum().item()
    accuracy = correct / mask.sum().item()
    return accuracy, loss.item()

In [22]:
devcie = torch.device('cpu' if torch.cuda.is_available() else 'cpu')



In [ ]:
model = GCN()

In [3]:
import numpy as np
from collections import defaultdict
import scipy.sparse as sp
import torch as th
from sklearn.preprocessing import OneHotEncoder
from torch_geometric.data import HeteroData

data_folder = "data/"


def encode_onehot(labels):

    # reshapes the numpy array to one column and whatever rows is required. 
    labels = labels.reshape(-1, 1)
    # print(f'labels inside encode_oneshot {labels}')

    # tranfromss the labels or categorical array into a matrix of 0 and 1 that encodes where the data is presenst
    # this is used to pass in models for understanding where the catgories are. 
    enc = OneHotEncoder()

    #This method is used to fit the encoder to the data, learning the unique categories for each feature that will
    #  be transformed during the encoding process.
    enc.fit(labels)

    # converst categoriacal data into a binary matrix. 
    labels_onehot = enc.transform(labels).toarray()

    # returns this oneshot binary matrix. 
    return labels_onehot


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""

    # gets a matrix plugged in and sums up all the rows of each the matrix and stores them in a tensor which gest turned into an array. 
    rowsum = np.array(features.sum(1))


    r_inv = np.power(rowsum, -1).flatten() # this performs an element-wise inverse on rowsum and then flatten the results to a 1-d array
    r_inv[np.isinf(r_inv)] = 0. # checks if any of the values are infinity prompting them to be equal to zero. 
    r_mat_inv = sp.diags(r_inv) # construct a digonal sparse matrix using the array of r_inv
    features = r_mat_inv.dot(features) # this multiples the new digonal matrix by the original features matrix. 
    if isinstance(features, np.ndarray):
        return features # if features was a numpy array it returns the new matrix 
    else:
        return features.todense() # if features matrix is not a numpy array it turns it into a dense matrix


def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""


    # the adj is a matrix that tells us which nodes are connecte dwith each other. 
    adj = sp.coo_matrix(adj) # this turns the adj matrix into a coo matrix which is used to save memory and better fro computation 
    # only saves the none zero objects in the matrix. 
    # print(adj)
    rowsum = np.array(adj.sum(1))# sums up each row in the adj matrix and makes them into a numpy array
    d_inv_sqrt = np.power(rowsum, -0.5).flatten() # elemnt-wise inverse square root from rowsum, then flattens it to a 1-d array
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0. # checks to see if any values infinity if so it turns it into zero 
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt) # creates a sparse diagonal matrix where the values of d_inv_sqrt are placed in the diagonal 
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo() # perfroms the symmetric normalization of the adjacency matrix. 


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = th.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = th.from_numpy(sparse_mx.data)
    shape = th.Size(sparse_mx.shape)
    return th.sparse.FloatTensor(indices, values, shape)



def process_data_in_pyg(neigs):
    d = defaultdict(dict)
    metapaths = []
    for mp_i, nei1 in enumerate(neigs):
        dst_array_concat = np.concatenate(nei1)
        src_array_concat = []
        for src_id, dst_array in enumerate(nei1):
            src_array_concat.extend([src_id] * len(dst_array))
        src_array_concat = np.array(src_array_concat)
        src_name = f"target"
        dst_name = f"dst_{mp_i}"
        relation = f"relation_{mp_i}"
        d[(src_name, relation + "-->", dst_name)]["edge_index"] = th.LongTensor(np.vstack([src_array_concat, dst_array_concat]))
        metapaths.append((src_name, relation + "-->", dst_name))
        d[(dst_name, "<--" + relation, src_name)]["edge_index"] = th.LongTensor(np.vstack([dst_array_concat, src_array_concat]))
        metapaths.append((dst_name, "<--" + relation, src_name))
    g = HeteroData(d)
    return g, metapaths



def load_aminer(ratio, type_num):
    # The order of node types: 0 p 1 a 2 \r

    # creates the path to aminer dataset 
    path = data_folder + "aminer/"

    #loads the labels.npy into a np array 
    label = np.load(path + "labels.npy").astype('int32')
    # print(f'lable:{label}')


    label = encode_onehot(label)
    # print(f'lable after encode oneshot {label}')
    # load object that are seralized inside a numpy array 
    # these objects are from the authors. 
    nei_a = np.load(path + "nei_a.npy", allow_pickle=True)\

    # load object that are searilized inside a numpy array 
    # this is for relationships. 
    nei_r = np.load(path + "nei_r.npy", allow_pickle=True)

    # Because none of P, A or R has features, we assign one-hot encodings to all of them.
    # the fatures are added to the papers, authors and relationships 
    # the type-num is "type_num": [6564, 13329, 35890],
    # so paper -> 6564, author -> 13329 -> relationships -> 35890
    # spicy.sparse.eye create a identity matrix of size type_num 
    feat_p = sp.eye(type_num[0]) #return a tuple of the indices connected witht he value for example (2,2) 1, means there is a 1 in row 2, column2, this goes down to the size of the array. 

    feat_a = sp.eye(type_num[1])
    feat_r = sp.eye(type_num[2])

    #loads sparse matrix that have been stored in paper-author-paper, paper-relationship-paper, paper-
    pap = sp.load_npz(path + "pap.npz")
    prp = sp.load_npz(path + "prp.npz")
    pos = sp.load_npz(path + "pos.npz")

    # contains the indices of the nodes for the train, test, val
    # .npy is a way to store numpy arrays into files
    train = [np.load(path + "train_" + str(i) + ".npy") for i in ratio]
    test = [np.load(path + "test_" + str(i) + ".npy") for i in ratio]
    val = [np.load(path + "val_" + str(i) + ".npy") for i in ratio]


    label = th.FloatTensor(label) # make label a float tensor 
    nei_a = [th.LongTensor(i) for i in nei_a] # make neighboring area a tensor instead of a numpy array
    nei_r = [th.LongTensor(i) for i in nei_r] # make neighboring area of relationships a float tensor intstead of a numpy array. 

    # go into the preporccess features. 
    # print("")
    # print(feat_a, feat_a, feat_r)

    # passes the feat_a p, r sparse matrices with are idneitity matrices. there are tuples designenating row and column with a value next to it 
    # representing the value that is in the matrix. 
    feat_p = th.FloatTensor(preprocess_features(feat_p)) # this turns the matrix that is returned int a float tensor 
    # print('')
    # print(feat_p)
    feat_a = th.FloatTensor(preprocess_features(feat_a))
    feat_r = th.FloatTensor(preprocess_features(feat_r))

    # print('')
    # print(pap)
    pap = sparse_mx_to_torch_sparse_tensor(normalize_adj(pap)) # perfroms a symetric normalization on the matrices. _
    prp = sparse_mx_to_torch_sparse_tensor(normalize_adj(prp))
    pos = sparse_mx_to_torch_sparse_tensor(pos)
    train = [th.LongTensor(i) for i in train]
    val = [th.LongTensor(i) for i in val]
    test = [th.LongTensor(i) for i in test]
    # print(train)
    return [nei_a, nei_r], [feat_p, feat_a, feat_r], [pap, prp], pos, label, train, val, test

def load_data(ratio=[20,40,60], type_num=[6564, 13329, 35890]):
    data = load_aminer(ratio, type_num)
    g, metapaths = process_data_in_pyg(data[0])
    return data, g, metapaths


(nei_index, feats, mps, pos, label, idx_train, idx_val, idx_test), g, metapaths = load_data()

C:\Users\16822\AppData\Local\Temp\ipykernel_30484\3511987151.py:71: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:620.)
  return th.sparse.FloatTensor(indices, values, shape)


In [47]:
pos

tensor(indices=tensor([[   0,  124,  168,  ..., 6536, 6547, 6563],
                       [   0,    0,    0,  ..., 6563, 6563, 6563]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(6564, 6564), nnz=67730, layout=torch.sparse_coo)

In [54]:
a, b , r = feats
sum = a.shape[0] + b.shape[0] + r.shape[0]
print(sum)
print(label.shape)

55783
torch.Size([6564, 4])


In [52]:
h = torch.cat((torch.cat(idx_train), torch.cat(idx_test), torch.cat(idx_val)), dim = 0)
h.shape

torch.Size([6480])

In [24]:
from torch_geometric.data import Data

def pad_feats(feat_matrix, target_matrix):
    '''This function pads the feature matrix with zeros in order to allow for concatination'''
    if feat_matrix.shape[1] < target_matrix[1]:
        padding = torch.zeros(feat_matrix.shape[0], target_matrix[1] - feat_matrix.shape[1])
        feat_matrix = torch.cat([feat_matrix, padding], dim=1)
    return feat_matrix

def homogenous_graph(nei_index, feats, mps, pos, label, idx_train, idx_val, idx_test):
    '''Using the outputs of load_aminer given in load_data.py file from HGMAE. This construst a 
    homogenous graph.'''

    # get features 
    feat_papers, feat_authors, feat_relationships = feats
    target_shape = (max(feat_papers.shape[0], feat_authors.shape[0], feat_relationships.shape[0]), 
                    max(feat_papers.shape[1], feat_authors.shape[1], feat_relationships.shape[1]))
    
    # pad the features 
    feat_papers = pad_feats(feat_papers, target_shape)
    feat_authors = pad_feats(feat_authors, target_shape)
    feat_relationships = pad_feats(feat_relationships, target_shape)

    # create features matrix 
    features = torch.cat([feat_papers, feat_authors, feat_relationships], dim =0)
    # print(features)
    # print(features.shape)

    pap, prp = mps
    node_papers, node_authors, node_relationships = feat_papers.shape[0], feat_authors.shape[0], feat_relationships.shape[0]

    pap_edge_index = pap.coalesce().indices()
    prp_edge_index = prp.coalesce().indices()
    # print(pap)
    # print('')
    # print(pap_edge_index)
    # print('')
    # print(prp, prp_edge_index)

    pap_edge_index += node_papers
    prp_edge_index += node_papers + node_relationships

    edge_index = torch.cat((pap_edge_index, prp_edge_index), dim = 1)

    # print(edge_index.shape)

    data = Data(x=features, edge_index=edge_index, y= label)
    num_nodes = data.num_nodes
    # print(num_nodes)

    train_mask, val_mask, test_mask = torch.zeros(num_nodes, dtype= torch.bool), torch.zeros(num_nodes, dtype=torch.bool), torch.zeros(num_nodes, dtype=torch.bool)

    # print(train_mask.shape)
    idx_train = torch.cat(idx_train)
    idx_val = torch.cat(idx_val)
    idx_test = torch.cat(idx_test)
    # print(idx_test.shape[0] + idx_val.shape[0] + idx_train.shape[0])

    train_mask[idx_train] = True
    val_mask[idx_val] = True
    test_mask[idx_test] = True 

    data.train_mask, data.val_mask, data.test_mask = train_mask, val_mask, test_mask

    data.num_classes = data.y.shape[1]

    return data

    # print(train_mask)



data = homogenous_graph(nei_index, feats, mps, pos, label, idx_train, idx_val, idx_test)

In [10]:

data.num_nodes

55783

In [20]:
data.y.shape


torch.Size([6564, 4])

In [23]:
data.num_features

35890

In [25]:
data.num_classes

4

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [27]:
import torch.optim as optim

In [37]:

model = GCN(data.num_features, hidden_channels=256, num_classes=data.num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

best_val_acc = 0.0
best_model_state = None

for epoch in range(301):
    train_loss = train(model, optimizer, data, data.train_mask)
    val_acc, val_loss = evaluate(model, data, data.val_mask)
    test_acc, test_loss = evaluate(model, data, data.test_mask)

    if epoch % 200 == 0:
    
        print(f"Epoch: {epoch + 1}/300 | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f} | Test Acc: {test_acc:.4f}")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict()

IndexError: The shape of the mask [55783] at index 0 does not match the shape of the indexed tensor [6564, 4] at index 0